In [1]:
import json
import numpy as np
from bitarray import bitarray
from array import *

In [2]:
with open('CNN_info_Sample.json', 'r') as f:
    CNN_data = json.load(f)
layers = len(CNN_data['layers'])

In [3]:
ak = array('B')
bin((int("00000011", 2) - (1 << 8)))[3:]

'11111101'

In [4]:
my_int = 11523
bin(my_int)[2:].zfill(16)

'0010110100000011'

In [5]:
def split(big_int):
    little_endian = big_int % 256
    big_endian = (big_int >> 8)
    return little_endian, big_endian

In [6]:
def twos_compliment(quantized_string):
    return bin((int(quantized_string, 2) - (1 << 8)))[3:11]

In [7]:
def makeFixedStrings(number, integers,floating):
    curr = number
    string = ''
    string += '0'
    neg = False
    if curr < 0:
        neg = True
    curr = abs(curr)
    for i in range(integers-2,-1,-1):
        if(curr > np.power(2,i,dtype = float)):
            curr -= np.power(2,i,dtype = float)
            string += '1'
        else:
            string += '0'
    curr -= int(curr)
    for i in range(1,floating+1):
        if(curr > np.power(2,-i,dtype = float)):
            curr -= np.power(2,-i,dtype = float)
            string +='1'
        else:
            string +='0'
    #print(string)
    if neg:
        string = twos_compliment(string)
    return string

In [8]:
makeFixedStrings(0.07902283770829079, 2, 6)
bin(int('00000100', 2) - (1 << 8))[3:11]

'00000101'

'11111100'

In [9]:
def appendConv(layer, CNN_inputs, CNN_inputs_bitstring, CNN_inputs_binary, CNN_count):
    biases = list(layer['biases']['w'].values())
    for i, conv_filter in enumerate(layer['filters']):
        CNN_inputs.append("bias of filter " + str(i) + ": " + str(biases[i]))
        quantized_bias = makeFixedStrings(biases[i], 2, 6)
        CNN_inputs_binary.append(int(quantized_bias, 2))
        CNN_inputs_bitstring.append(quantized_bias)
        for weight in conv_filter['w'].values():
            CNN_inputs.append(weight)
            quantized_weight = makeFixedStrings(weight, 2, 6)
            CNN_inputs_binary.append(int(quantized_weight, 2))
            CNN_inputs_bitstring.append(quantized_weight)
    #return (len(layer['filters']) * len(layer['filters'][0]['w'].values())) + len(layer['filters'])

In [10]:
def appendFC(layer, FC_inputs, FC_inputs_bitstring, FC_inputs_binary):
    num_inputs = layer['num_inputs'] + 1
    FC_inputs.append("FC input number: " + str(num_inputs))
    #Padding since RAM has width of 10 bytes
    for k in range(2): #1 for global counter, 1 for Kaseb's byte
        little, big = split(num_inputs)
        FC_inputs_binary.append(int(big))
        FC_inputs_binary.append(int(little))
        FC_inputs_bitstring.append(bin(big)[2:].zfill(8))
        FC_inputs_bitstring.append(bin(little)[2:].zfill(8))
    
    
    for i in range(8):  #3 empty words + 1 for total inputs + 1 for depth = 5 (Ram width in 16-bit words)
        FC_inputs_binary.append(int(0))
        FC_inputs_bitstring.append(makeFixedStrings(0, 2, 6))
                     
    FC_weights = np.zeros((10, num_inputs))
    biases = list(layer['biases']['w'].values())
    for i, bias in enumerate(biases):
        FC_inputs.append("bias " + str(i) + ": " + str(bias))
        quantized_bias = makeFixedStrings(bias, 2, 6)
        FC_inputs_binary.append(int(quantized_bias, 2))
        FC_inputs_bitstring.append(quantized_bias)
    for i, FC_filter in enumerate(layer['filters']):
        weights = np.zeros(num_inputs)
        for j, weight in enumerate(FC_filter['w'].values()):
            weights[j] = weight
        FC_weights[i,:] = weights
    for col_num in range(num_inputs - 1):
        for FC_weight in FC_weights[:, col_num]:
            quantized_FC_weight = makeFixedStrings(FC_weight, 2, 6)
            FC_inputs_binary.append(int(quantized_FC_weight, 2))
            FC_inputs_bitstring.append(quantized_FC_weight)
            FC_inputs.append(FC_weight)

In [11]:
CNN_inputs = [0, 0, layers]
CNN_inputs_bitstring = [0, 0, layers]
CNN_inputs_binary = array('B')
CNN_inputs_binary.append(int(0))
CNN_inputs_binary.append(int(0))
CNN_inputs_binary.append(int(layers))
FC_inputs = []
FC_inputs_bitstring = []
FC_inputs_binary = array('B')
config_layers = 0
for layer in CNN_data['layers']:
    print('--------------------------')
    layer_type = layer['layer_type']
    print(layer_type)
    if layer_type == 'input' or layer_type == 'relu' or layer_type == 'softmax':
        print('Malhash lazma')
        continue
    if layer_type == 'fc':
        appendFC(layer, FC_inputs, FC_inputs_bitstring, FC_inputs_binary)
        continue
    config_layers += 1    #Only pool or conv layers need config word
    config_word = bitarray(2**4)  #Bit array of size 16
    config_word.setall(0)   #Resets the bit array to zeros
    depth = layer['out_depth']
    out_dim = layer['out_sx']
    depth_bits = bitarray([depth >> i & 1 for i in range(3,-1,-1)])  #loops from 3 to 0, step of -1
    dim_bits = bitarray([out_dim >> i & 1 for i in range(4,-1,-1)])
    config_word[2: 6] = config_word[2: 6] | depth_bits
    config_word[8 : 13] = config_word[8 : 13] | dim_bits
    sx = layer['sx']
    sy = layer['sy']
    if sx == 3 and sy == 3:
        print('Filter size is 3')
        config_word[1] = 0
    elif sx == 5 and sy == 5:
        print('Filter size is 5')
        config_word[1] = 1
    if layer_type == 'pool':
        config_word[0] = 0
    if layer_type == 'conv':
        config_word[0] = 1
    CNN_inputs.append(config_word)
    config_int = int(config_word.to01(), 2)
    little, big = split(config_int)
    CNN_inputs_binary.append(int(big))  #Left part of configuration word
    CNN_inputs_binary.append(int(little))  #Right part
    CNN_inputs_bitstring.append(bin(big)[2:].zfill(8))
    CNN_inputs_bitstring.append(bin(little)[2:].zfill(8))
    if layer_type == 'conv':
        appendConv(layer, CNN_inputs, CNN_inputs_bitstring, CNN_inputs_binary, CNN_count)
            
CNN_count = len(CNN_inputs) - 2
if CNN_count % 2 == 1:
    CNN_inputs.append(0)
    CNN_inputs_binary.append(int(0))
    CNN_inputs_bitstring.append(makeFixedStrings(0, 2, 6))
CNN_count = int((CNN_count + 1) / 2)
little, big = split(CNN_count)
CNN_inputs_binary[0] = int(big)
CNN_inputs_binary[1] = int(little)
CNN_inputs_bitstring[0] = bin(big)[2:].zfill(8)
CNN_inputs_bitstring[1] = bin(little)[2:].zfill(8)

CNN_inputs[2] = config_layers
CNN_inputs_binary[2] = int(config_layers)
CNN_inputs_bitstring[2] = bin(config_layers)[2:].zfill(8)

--------------------------
input
Malhash lazma
--------------------------
conv
Filter size is 5


NameError: name 'CNN_count' is not defined

In [ ]:
f = open('FC.binary','wb')
FC_inputs_binary.tofile(f)
f.close()

In [ ]:
f = open('CNN.binary','wb')
CNN_inputs_binary.tofile(f)
f.close()

In [ ]:
f = open('input.binary','wb')
CNN_inputs_binary.tofile(f)
FC_inputs_binary.tofile(f)
f.close()

In [ ]:
with open('CNN_input.txt', 'w') as file:
    for i in range(len(CNN_inputs)):
        file.write(str(CNN_inputs[i]))
        file.write('\n')
file.close()

In [ ]:
'''
with open('FC_input.txt', 'w') as file:
    for i in range(len(FC_inputs)):
        file.write(str(FC_inputs[i]))
        file.write('\n')
file.close()
'''

In [ ]:
with open('CNN_input_bitstring.txt', 'w') as file:
    count = 0
    for i in range(len(CNN_inputs_bitstring)):
        if count == 2:
            file.write('\n')
            count = 0
        file.write(str(CNN_inputs_bitstring[i]))
        file.write(' ')
        count += 1
file.close()

In [ ]:
'''
with open('FC_input_bitstring.txt', 'w') as file:
    file.write(FC_inputs_bitstring[0])
    file.write(FC_inputs_bitstring[1])
    file.write('\n')
    count = 0
    for i in range(2, len(FC_inputs_bitstring)):
        if count == 10:
            file.write('\n')
            count = 0
        file.write(str(FC_inputs_bitstring[i]))
        file.write(' ')
        count += 1
file.close()
'''

In [ ]:
'''
with open('input.txt', 'w') as file:
    for i in range(len(CNN_inputs)):
        file.write(str(CNN_inputs[i]))
        file.write('\n')
    for i in range(len(FC_inputs)):
        file.write(str(FC_inputs[i]))
        file.write('\n')
'''
